In [2]:
from glob import glob

base_path = '/mnt/datasets/ben/backup/raid/SubtleGAD_Stanford/lowcon'
data_path = '/home/srivathsa/projects/studies/gad/stanford/preprocess/data_256_fb'
all_cases = sorted([c.split('/')[-1] for c in glob('{}/Patient*'.format(base_path))])
cases = sorted([c.split('/')[-1].replace('.npy', '') for c in glob('{}/*.npy'.format(data_path))])

bpath_siemens = '/mnt/datasets/ben/backup/raid/SubtleGAD_Stanford/Siemens'
siemens_cases = sorted([c.split('/')[-1] for c in glob('{}/*'.format(bpath_siemens))])

rem_cases = [c for c in all_cases if c not in cases and c not in siemens_cases]

print(rem_cases)

['Patient_0094', 'Patient_0100', 'Patient_0111', 'Patient_0163', 'Patient_0190', 'Patient_0203', 'Patient_0564']


In [3]:
import os
from subtle.utils.io import get_dicom_dirs, get_dcmdir_with_kw
from tqdm import tqdm_notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (12, 10)

data_path = '/home/srivathsa/projects/studies/gad/stanford/data'
test_data_path = '/home/srivathsa/projects/studies/gad/stanford/preprocess/data_mmt/full_brain_256/test'
test_cases = sorted([c.split('/')[-1] for c in glob('{}/*'.format(test_data_path))])
dest_path = '/home/srivathsa/projects/studies/gad/stanford/data/mdi_test'

for cnum in tqdm(test_cases, total=len(test_cases)):
    case_path = os.path.join(data_path, cnum)
    fp_t1, _, fp_t1c = get_dicom_dirs(case_path)
    fp_t2 = get_dcmdir_with_kw(case_path, kws=['ax', 't2'])
    fp_fl = get_dcmdir_with_kw(case_path, kws=['flair'])
    
    print(cnum, fp_fl)
    dest_cpath = os.path.join(dest_path, cnum)
#     os.makedirs(dest_cpath, exist_ok=True)
    
#     os.symlink(fp_t1, os.path.join(dest_cpath, fp_t1.split('/')[-1]))
#     os.symlink(fp_t1c, os.path.join(dest_cpath, fp_t1c.split('/')[-1]))
#     os.symlink(fp_t2, os.path.join(dest_cpath, fp_t2.split('/')[-1]))
#     os.symlink(fp_fl, os.path.join(dest_cpath, fp_fl.split('/')[-1]))
    

<ipython-input-3-0b694d92b4ac>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for cnum in tqdm(test_cases, total=len(test_cases)):


  0%|          | 0/49 [00:00<?, ?it/s]

Patient_0010 /home/srivathsa/projects/studies/gad/stanford/data/Patient_0010/400_AX_CUBE_FLAIR
Patient_0022 /home/srivathsa/projects/studies/gad/stanford/data/Patient_0022/500_AX_FLAIR_CUBE
Patient_0049 /home/srivathsa/projects/studies/gad/stanford/data/Patient_0049/400_AX_CUBE_FLAIR_REFORMAT
Patient_0068 /home/srivathsa/projects/studies/gad/stanford/data/Patient_0068/400_AX_CUBE_FLAIR_REFORMAT
Patient_0071 /home/srivathsa/projects/studies/gad/stanford/data/Patient_0071/400_AX_CUBE_FLAIR
Patient_0078 /home/srivathsa/projects/studies/gad/stanford/data/Patient_0078/500_AX_CUBE_FLAIR
Patient_0080 /home/srivathsa/projects/studies/gad/stanford/data/Patient_0080/600_AX_FLAIR_CUBE
Patient_0091 /home/srivathsa/projects/studies/gad/stanford/data/Patient_0091/12_Sag_SPACE_FLAIR_MPR_Ax_Reformat
Patient_0115 /home/srivathsa/projects/studies/gad/stanford/data/Patient_0115/400_AX_FLAIR_CUBE
Patient_0117 /home/srivathsa/projects/studies/gad/stanford/data/Patient_0117/400_AX_CUBE_FLAIR
Patient_0146 /h

<Figure size 864x720 with 0 Axes>

In [ ]:
kw_str = {
    'T1': 'AX_BRAVO',
    'T1CE': 'AX_BRAVO_+C',
    'T2': 'AX_T2',
    'FLAIR': 'AX_CUBE_FLAIR'
}

for cnum in rem_cases:
    dcm_dirs = [d for d in glob('{}/{}/*'.format(base_path, cnum))]
    has_series = []
    for k, v in kw_str.items():
        kw_ser = [s for s in dcm_dirs if v.lower() in s.lower()]
        has_series.append(len(kw_ser) > 0)
    print(cnum, has_series)

### Move `lowcon` cases to stanford/data

In [ ]:
import os
from glob import glob
import pydicom
from tqdm.notebook import tqdm
import shutil

def get_ser_desc(dpath_dcm):
    try:
        dcm_files = [f for f in glob('{}/*.dcm'.format(dpath_dcm))]
        if len(dcm_files) == 0:
            return ''
        dcm = pydicom.dcmread(dcm_files[0])

        return '{}_{}'.format(dcm.SeriesNumber, dcm.SeriesDescription.replace(' ', '_').replace('/', '_'))
    except Exception:
        return ''

def str_has_kws(kws_str, desc_str):
    for kw in kws_str:
        if kw.lower() not in desc_str.lower():
            return False
    return True

def find_series_with_kws(desc_map, kws, exclude=None):
    for k, v in desc_map.items():
        if k == exclude: continue
        if str_has_kws(kws, k):
            return (k, v)
    return None

src_path = '/mnt/datasets/ben/backup/raid/SubtleGAD_Stanford/lowcon'
cases = sorted([c.split('/')[-1] for c in glob('{}/*'.format(src_path)) if 'lowcon' in c or 'Patient' in c])
# dest_path = '/mnt/datasets/srivathsa/sford_extra'
# proc_cases = sorted([c.split('/')[-1].replace('Patient', 'lowcon') for c in glob('{}/*'.format(dest_path))])
# cases = [c for c in cases if c not in proc_cases]

kw_map = {
    'T1': ['ax', 'bravo'],
    'T1_low': ['ax', 'bravo', '+c'],
    'T1_full': ['ax', 'bravo', '+c'],
    'T2': ['ax', 't2'],
    'FLAIR': ['ax', 'cube', 'flair']
}

for cnum in tqdm(cases, total=len(cases)):
    pdir = os.path.join(src_path, cnum)
    dicom_sers = sorted([d for d in glob('{}/*'.format(pdir)) if os.path.isdir(d)])
    desc_map = {get_ser_desc(dpath): dpath for dpath in dicom_sers}
    mra_sers = find_series_with_kws(desc_map, ['cow'])
    if mra_sers is not None:
        print(cnum)
#     case_map = {}
#     pid = cnum.replace('lowcon', 'Patient')
#     dest_dir = os.path.join(dest_path, pid)
#     os.makedirs(dest_dir, exist_ok=True)

#     for k, v in kw_map.items():
#         if k == 'T1_full':
#             case_map[k] = find_series_with_kws(desc_map, kw_map[k], exclude=case_map['T1_low'][0])
#         else:
#             case_map[k] = find_series_with_kws(desc_map, kw_map[k])
#         if case_map[k] is not None:
#             desc, dirpath = case_map[k]
#             shutil.copytree(dirpath, os.path.join(dest_dir, desc))

In [6]:
import nibabel as nib
import matplotlib.pyplot as plt
import subtle.utils.io as suio

plt.set_cmap('gray')

data = nib.load(
    '/mnt/datasets/srivathsa/jiang_raid/projects/SubtleGAN/data/IXI/IXI_MRA/coreg/IXI098-Guys-0745/IXI098-Guys-0745-T1.nii.gz'
).get_fdata()
print(data.shape)

/home/srivathsa/projects/SubtleGad/train/subtle/utils/hyperparameter.py:8: UserWarning: Module test_tube not found - hyperparameter related functions cannot be used
  warnings.warn('Module test_tube not found - hyperparameter related functions cannot be used')


(512, 512, 100)


<Figure size 432x288 with 0 Axes>

In [20]:
from glob import glob
import subtle.utils.experiment as su_exp

exp_cases = su_exp.get_experiment_data('mra_synth', '/home/srivathsa/projects/SubtleGad/train/configs/experiments')
dcm_cases = sorted([
    c.split('/')[-1] for c in 
    glob('/mnt/datasets/srivathsa/jiang_raid/projects/SubtleGAN/data/IXI/IXI_MRA/dcms/IXI*')
])

rem_cases = sorted([c for c in dcm_cases if c not in exp_cases])

rem_cases

['IXI184-Guys-0794',
 'IXI185-Guys-0795',
 'IXI186-Guys-0796',
 'IXI188-Guys-0798',
 'IXI189-Guys-0799',
 'IXI192-Guys-0878',
 'IXI193-Guys-0810',
 'IXI194-Guys-0818',
 'IXI195-HH-1620',
 'IXI196-Guys-0805',
 'IXI197-Guys-0811',
 'IXI198-Guys-0803',
 'IXI199-Guys-0802',
 'IXI200-Guys-0812',
 'IXI201-HH-1588',
 'IXI202-HH-1526',
 'IXI204-HH-1651',
 'IXI205-HH-1649',
 'IXI206-HH-1650',
 'IXI207-Guys-0809',
 'IXI208-Guys-0808',
 'IXI209-Guys-0804',
 'IXI210-Guys-0856',
 'IXI211-HH-1568',
 'IXI212-HH-1643',
 'IXI214-HH-1636',
 'IXI216-HH-1635',
 'IXI217-HH-1638',
 'IXI218-HH-1815',
 'IXI219-Guys-0894',
 'IXI221-HH-1606',
 'IXI222-Guys-0819',
 'IXI223-Guys-0830',
 'IXI224-Guys-0823',
 'IXI227-Guys-0813',
 'IXI228-Guys-0822',
 'IXI230-IOP-0869',
 'IXI231-IOP-0866',
 'IXI232-IOP-0898',
 'IXI234-IOP-0870',
 'IXI236-Guys-0817',
 'IXI237-Guys-1049',
 'IXI238-IOP-0883',
 'IXI239-HH-2296',
 'IXI240-Guys-0834',
 'IXI241-Guys-0833',
 'IXI242-HH-1722',
 'IXI244-Guys-0841',
 'IXI246-Guys-0840',
 'IXI2